# Morphosyntax

In [1]:
import nltk

Download corpora:
- Macmorpho
- Floresta

In [2]:
# nltk.download()

In [3]:
# Floresta: corpus sintatico, Mac_Morpho: morfosintatico
from nltk.corpus import mac_morpho, floresta

# Mac_morpho

In [4]:
tagged_sentences = []

for sentence in mac_morpho.tagged_sents():
    tagged_sentences.append(sentence)
print(len(tagged_sentences))

51397


# Floresta

In [5]:
tagged_sentences = []
def simplify_tag_floresta(t):
    if "+" in t:
        return t[t.index("+")+1:].upper()
    else:
        return t.upper()

for sentence in floresta.tagged_sents():
    sentence = [(text, simplify_tag_floresta(tag)) for text, tag in sentence]
    tagged_sentences.append(sentence)
print(len(tagged_sentences))

9266


In [5]:
from collections import Counter

all_tags = []
for sent in tagged_sentences:
    for w, t in sent:
        all_tags.append(t)

tagset_freq = Counter(all_tags)
tagset_freq

Counter({'!': 919,
         '"': 21069,
         '$': 60,
         "'": 510,
         '(': 7713,
         '((': 10,
         ')': 7741,
         '))': 10,
         ',': 68494,
         '-': 3117,
         '.': 82,
         '...': 84,
         '/': 95,
         ':': 6736,
         ';': 1376,
         '=': 14,
         '?': 1510,
         'ADJ': 53372,
         'ADJ|+': 11,
         'ADJ|EST': 174,
         'ADV': 30653,
         'ADV-KS': 392,
         'ADV-KS-REL': 898,
         'ADV|+': 8,
         'ADV|EST': 9,
         'ADV|HOR': 9,
         'ADV|[': 18,
         'ADV|]': 13,
         'ART': 151891,
         'ART|+': 7,
         'ART|EST': 4,
         'CUR': 2706,
         'IN': 415,
         'IN|EST': 1,
         'KC': 28262,
         'KC|+': 2,
         'KC|EST': 2,
         'KC|[': 143,
         'KC|]': 133,
         'KS': 14320,
         'KS|[': 5,
         'KS|]': 4,
         'N': 236462,
         'NPRO': 3,
         'NPROP': 114318,
         'NPROP|+': 35,
         'NUM': 1811

- Pode ser feito um tratamento para reduzir o número de classes e melhorar a acuracia
- Metricas de avaliação:
    - Precisão: do que eu classifiquei quanto eu acertei
    - Cobertura: de todo o conjunto quanto eu classifiquei correto
    - F-Measure: Media armonica das duas anteriores

# Training a tagger

In [8]:
# Embaralhar pode melhorar principalmente se o floresta e o mac_morfo forem concatenados

cutoff = int(.80 * len(tagged_sentences))
training_sentences = tagged_sentences[:cutoff]
test_sentences = tagged_sentences[cutoff:]
 
print(len(training_sentences))
print(len(test_sentences))

41117
10280


# Always a noun

In [9]:
# Classifica todo mundo como N => Dummy classifier => Classe majoritaria
tagger_default = nltk.DefaultTagger('N')
tagger_default.evaluate(test_sentences)

0.19676506865571755

# Most frequent tag

In [10]:
# Recebe a etiqueta mais comum que apresenta a palavras

unigram_tagger = nltk.UnigramTagger(training_sentences, verbose=True, backoff=tagger_default)
unigram_tagger.evaluate(test_sentences)

[Trained Unigram tagger: size=43530, backoff=25.59%, pruning=23.91%]


0.8156664993560486

# Bigram tagger

In [12]:
# Olha a janela de palavras antes e depois: no caso 2 palavras

bigram_tagger = nltk.BigramTagger(training_sentences, verbose=True, backoff=unigram_tagger)
bigram_tagger.evaluate(test_sentences)

[Trained Unigram tagger: size=12515, backoff=56.22%, pruning=90.08%]


0.8298100829938484

# Trigram tagger

In [13]:
# Janela de 3 palavras

trigram_tagger = nltk.TrigramTagger(training_sentences, verbose=True, backoff=bigram_tagger)
trigram_tagger.evaluate(test_sentences)

[Trained Unigram tagger: size=12937, backoff=66.56%, pruning=94.25%]


0.8297011365449191

# Tagging

In [17]:
# Do = de + o ele classifica certo, aconcelhavel fazer isso como proprocessamento

sentence = "Uma vez que você tenha aceitado suas falhas, ninguém poderá usá-las contra você".split()
print("Default")
tagged_sentence = tagger_default.tag(sentence)
print(tagged_sentence)
print("Unigram")
tagged_sentence = unigram_tagger.tag(sentence)
print(tagged_sentence)
print("Bigram")
tagged_sentence = bigram_tagger.tag(sentence)
print(tagged_sentence)
print("Trigram")
tagged_sentence = trigram_tagger.tag(sentence)
print(tagged_sentence)

Default
[('Uma', 'N'), ('vez', 'N'), ('que', 'N'), ('você', 'N'), ('tenha', 'N'), ('aceitado', 'N'), ('suas', 'N'), ('falhas,', 'N'), ('ninguém', 'N'), ('poderá', 'N'), ('usá-las', 'N'), ('contra', 'N'), ('você', 'N')]
Unigram
[('Uma', 'ART'), ('vez', 'N'), ('que', 'PRO-KS-REL'), ('você', 'PROPESS'), ('tenha', 'VAUX'), ('aceitado', 'N'), ('suas', 'PROADJ'), ('falhas,', 'N'), ('ninguém', 'PROSUB'), ('poderá', 'VAUX'), ('usá-las', 'N'), ('contra', 'PREP'), ('você', 'PROPESS')]
Bigram
[('Uma', 'ART'), ('vez', 'N'), ('que', 'PRO-KS-REL'), ('você', 'PROPESS'), ('tenha', 'VAUX'), ('aceitado', 'N'), ('suas', 'PROADJ'), ('falhas,', 'N'), ('ninguém', 'PROSUB'), ('poderá', 'VAUX'), ('usá-las', 'N'), ('contra', 'PREP'), ('você', 'PROPESS')]
Trigram
[('Uma', 'ART'), ('vez', 'N'), ('que', 'PRO-KS-REL'), ('você', 'PROPESS'), ('tenha', 'VAUX'), ('aceitado', 'N'), ('suas', 'PROADJ'), ('falhas,', 'N'), ('ninguém', 'PROSUB'), ('poderá', 'VAUX'), ('usá-las', 'N'), ('contra', 'PREP'), ('você', 'PROPESS')]

In [20]:
# Salva o objeto treinado
import pickle

with open('bigram_tagger.pickle', 'wb') as p_file:
    p_file.write(pickle.dumps(bigram_tagger))

# Other corpus
http://www.nilc.icmc.usp.br/nilc/download/corpus100.txt

# Other taggers

In [23]:
import spacy

In [24]:
import subprocess
command = "python -m spacy download pt_core_news_sm".split()
subprocess.call(command)

0

In [25]:
nlp = spacy.load('pt_core_news_sm')
sentence = "O rato comeu a roupa do rei de roma"
doc = nlp(sentence)
for token in doc:
    print(token.text, token.pos_, token.tag_)

OSError: [E050] Can't find model 'pt_core_news_sm'. It doesn't seem to be a shortcut link, a Python package or a valid path to a data directory.

In [ ]:
import nlpnet

Download model from http://nilc.icmc.usp.br/nlpnet/data/pos-pt.tgz

In [ ]:
nlpnet.set_data_dir('pos-pt/')
sentence = "O rato comeu a roupa do rei de roma"
tagger = nlpnet.POSTagger()
tagger.tag(sentence)